In [0]:
!pip3 install sk-video

     |████████████████████████████████| 2.3MB 2.8MB/s 


In [0]:
import skvideo.io
import keras.models
from keras.layers import Dense, Dropout, Flatten
from keras.preprocessing import image
import numpy as np
import pandas as pd
from tqdm import tqdm
import cv2
import math
from keras.applications.vgg19 import VGG19
import os
from glob import glob
from scipy import stats
import skimage
from sklearn.metrics import accuracy_score

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cd /content/gdrive/My\ Drive/Data\ Sets

/content/gdrive/My Drive/Data Sets


In [0]:
max_nframes = 4
default_size = (112, 112, 3)
init_model = VGG19(weights='imagenet', include_top=False, input_shape = default_size)
model = keras.models.Sequential([
    Dense(756, activation='relu', input_dim=4608),
    Dropout(0.4),
    Dense(378, activation='relu'),
    Dropout(0.4),
    Dense(256, activation='relu'),
    Dropout(0.4),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(101, activation='softmax'),
])
model.load_weights("weights.hdf5")
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy', 'top_k_categorical_accuracy'])

In [0]:
f = open("ucfTrainTestlist/testlist01.txt", "r")
temp = f.read()
test = pd.DataFrame()
test['name'] = temp.split('\n')
test = test[:-1]
test_videos = test['name']

In [0]:
train = pd.read_csv('train_new.csv')
y = train['class']
y = pd.get_dummies(y)

In [0]:
predict, actual = [], []
for i in tqdm(range(test_videos.shape[0])):
    videoFile = test_videos[i]
    path = 'UCF-101/'+videoFile.split(' ')[0]
    cnt = 0
    z = skvideo.io.vreader(path, num_frames = max_nframes)
    cnt = 0
    prediction_images = []
    for i in z:
        prediction_images.append(skimage.transform.resize(np.squeeze(i[0]), default_size))
    prediction_images = init_model.predict(np.array(prediction_images))
    prediction_images = prediction_images.reshape((prediction_images.shape[0], 4608))
    prediction = model.predict_classes(prediction_images)
    predict.append(y.columns.values[stats.mode(prediction)[0][0]])
    actual.append(videoFile.split('/')[1].split('_')[1])

100%|██████████| 3783/3783 [51:19<00:00,  1.04s/it]


In [5]:
print(accuracy_score(predict, actual)*100)

14.538725878932064